In [72]:
import requests
import urllib
import unicodecsv as csv
from bs4 import BeautifulSoup
from datetime import datetime
import os


In [73]:
def get_topic_list_html(pagenum):
    r = requests.get('http://www.kdi.re.kr/policy/ep_list.jsp?pg='+str(pagenum)+'&&pp=10')
    
    soup = BeautifulSoup(r.text, "lxml")
    return soup

In [74]:
dataset = []
date = str(datetime.today()).split(" ")[0]
currentmonth =  date.split('-')[1]
currentday =  date.split('-')[2]

In [75]:
def get_data(html):
    keepgo = 1
    tmp_conts = html.find_all(class_="rpt_conts")[0]
    tmp_date = tmp_conts.find_all(class_="nlt")
    
    for i in range(0, len(tmp_date), 2):
        
        month = str(tmp_date[i]).split('.')[1]
        day = str(tmp_date[i]).split('.')[2]
        day = str(day).split('</')[0]
        
        j = (i/2)
      
        if month == currentmonth:
            if day == currentday:
                tmp_li = tmp_conts.find_all('a')
                tmp_li[j] = tmp_li[j].encode('utf-8')
                tmp_str = str(tmp_li[j]).split(">")[1]
                tmp_title = str(tmp_str).split("<")[0]

                
                tmp_li = tmp_conts.find_all(class_="rpt_sup")


                tmp_li[j] = tmp_li[j].encode('utf-8')
                tmp_str = str(tmp_li[j]).split("<em>")[1]
                tmp_ori = (tmp_str).split("</em>")[0]
                

                tmp_li = tmp_conts.find_all(class_="rpt_lst")


                tmp_str = str(tmp_li[j]).split(">")[1]
                tmp_str = str(tmp_str).split("\"")[3]
                tmp_str= str(tmp_str).split("./")[1]
                tmp_link = "www.kdi.re.kr/policy/" + tmp_str

                dataset.append({ '구분': '중앙부처', '발표처': tmp_ori, '제목': tmp_title, '웹주소': tmp_link})

            else:
                keepgo = 0
        else:
            keepgo = 0
            
    return keepgo

In [77]:
p = 1
keepgo = 1
while(keepgo == 1):
    
    test_html = get_topic_list_html(p)
    keepgo = get_data(test_html)
    p= p+1

In [78]:
def WriteDictToCSV(csv_file,csv_columns,dict_data):
    try:
        with open(csv_file, 'wb') as csvfile:
            csvfile.write(u'\ufeff'.encode('utf8'))
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in dict_data:
                writer.writerow({k:v for k,v in data.items()})
    except IOError as (errno, strerror):
            print("I/O error({0}): {1}".format(errno, strerror)) 
    return 


In [79]:
csv_columns = ["구분", "발표처", "제목", "웹주소"]
currentPath = os.getcwd()
csv_file = date + ".csv"
WriteDictToCSV(csv_file, csv_columns, dataset)
